<a href="https://colab.research.google.com/github/mitsuogui/Face-Recognition-Proyect-Talller/blob/main/II_ETAPA_ASISTENTE_AULAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instituto Tecnológico de Costa Rica
### Reconocimiento de Rostros
Author: [Johan Rodríguez](https://github.com/Johanx22x)

Note: When using this notebook, you must select a GPU Hardware Accelerator instead of CPU. Go to Runtime->Change Runtime Type and select any option that uses GPU.

# Dependencies

In [ ]:
import pickle #para guardar archivos

import os  #para guardar y manejar carpetas automatico


!pip install face_recognition         # Install face_recognition package using pip
import face_recognition
import cv2                            # Import OpenCV
import numpy as np
from matplotlib import pyplot as plt  # pyplot

# ipywidgets packages
import ipywidgets as widgets
from IPython.display import display, clear_output

# Live Streaming dependencies
from IPython.display import Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io
import html
import time

import pickle

# Utils

## Montar carpetas google

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Nombre de la carpeta que deseas crear
carpeta_nombre = "II E Asistente Aula"

# Ruta completa en tu Google Drive
carpeta_ruta = '/content/drive/My Drive/' + carpeta_nombre

# Comprueba si la carpeta ya existe, y si no, créala
if not os.path.exists(carpeta_ruta):
    os.mkdir(carpeta_ruta)
    print(f'Se ha creado la carpeta: {carpeta_nombre}')
else:
    print(f'La carpeta {carpeta_nombre} ya existe en tu Google Drive.')

# Nombre del archivo binario que deseas guardar
archivo_nombre = "mi_archivo.bin"

# Ruta completa del archivo binario
archivo_ruta = os.path.join(carpeta_ruta, archivo_nombre)

# Datos binarios con caracteres no ASCII (en este caso, caracteres en español)
datos_binarios = 'Tus datos binarios aquí con caracteres no ASCII: áéíóú'.encode('utf-8')

# Puedes abrir y escribir en el archivo binario
with open(archivo_ruta, "wb") as archivo_binario:
    archivo_binario.write(datos_binarios)

print(f'Se ha creado y guardado el archivo binario en: {archivo_ruta}')



Se ha creado la carpeta: II E Asistente Aula
Se ha creado y guardado el archivo binario en: /content/drive/My Drive/II E Asistente Aula/mi_archivo.bin


## In-memory data

In [ ]:
registered_faces = {}

## face recognition

In [ ]:
def check_if_face_registered(face_encoding):
  for _, registered_face in registered_faces.items():
    if True in face_recognition.compare_faces([face_encoding], registered_face):
      return True
  return False

## Real time streaming

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 800, 600);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 800; //video.videoWidth;
      captureCanvas.height = 600; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

In [ ]:
# JavaScript code to stop the webcam stream
js_stop_stream = """
    const stream = video.srcObject;
    const tracks = stream.getTracks();

    tracks.forEach(function(track) {
        track.stop();
    });

    video.srcObject = null;
    div.style.display = 'none';  // Hide the container
"""

# Function to stop the video streaming
def stop_video_stream():
    display(Javascript(js_stop_stream))

# Register Faces

In [ ]:
# Create a text input widget for the name
name_input = widgets.Text(
    placeholder='Enter your name',
    description='Name:',
    disabled=False
)

# Create a file upload widget
upload_button = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Upload Image'
)

or_text = widgets.HTML(
    value=" or "
)

# Create a button to trigger face registration
register_button = widgets.Button(description='Register Face')

# Create a button to trigger image capture
capture_button = widgets.Button(description='Capture Image')

# Create an output widget to display the uploaded or captured face image
output = widgets.Output()

# Initialize variables to store the uploaded image and name
uploaded_image_data = None
registered_name = None

# Define a function to handle image upload
def handle_upload(change):
    global uploaded_image_data
    uploaded_filename = list(upload_button.value.keys())[0]
    uploaded_image_data = upload_button.value[uploaded_filename]['content']

    with output:
        clear_output()
        display(widgets.Image(value=uploaded_image_data))

    # Assuming uploaded_image_data is binary image data in bytes
    image_array = np.frombuffer(uploaded_image_data, np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    # Convert the image to RGB format
    uploaded_image_data = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def handle_capture(button):
  global uploaded_image_data
  # start streaming video from webcam
  video_stream()
  js_reply = video_frame("Capturing...", "")

  stop_video_stream()  # Stop the video stream after capturing the frame

  if not js_reply:
    with output:
      clear_output()
      print("Error capturing image!")

  # convert JS response to OpenCV Image
  uploaded_image_data = js_to_image(js_reply["img"])

  with output:
    clear_output()
    # Encode the captured frame as PNG
    _, buffer = cv2.imencode('.png', uploaded_image_data)

    # Convert the encoded image to bytes
    image_bytes = buffer.tobytes()
    display(widgets.Image(value=image_bytes))

# Define a function to handle face registration
def handle_register(button):
    global registered_name

    if uploaded_image_data is not None:
        registered_name = name_input.value

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(uploaded_image_data)

        # If there is no face, raise a popup window
        if len(face_locations) == 0:
            with output:
              clear_output()
              print("No face detected in the image!")
              return

        # If there is more than one face, raise a popup window
        if len(face_locations) > 1:
            with output:
              clear_output()
              print("More than one face detected in the image!")
              return

        # Get the face encodings
        face_encodings = face_recognition.face_encodings(uploaded_image_data)

        # Check if the face is registered
        if check_if_face_registered(face_encodings[0]):
            with output:
              clear_output()
              print("Face already registered!")
              return

        with output:
            clear_output()
            print(f"Face registered for {registered_name}")
            registered_faces[registered_name] = face_encodings[0]
            #Crear instancia de Profesor corregir segun ejemplo ahora con nuevo nombre de funcion cambiarlo y guardarlo en el archivo
            #create_professor(registered_name)

upload_button.observe(handle_upload, names='value')
capture_button.on_click(handle_capture)
register_button.on_click(handle_register)

# Create a container for the widgets
container = widgets.VBox([name_input, widgets.HBox([upload_button, or_text, capture_button]), register_button, output])

# Display the container
display(container)
print(registered_faces)


{}


In [ ]:
print(registered_name)

Tony


In [ ]:
print(registered_faces)

In [ ]:
for person in registered_faces:
  print(type(person)," : ", person)

<class 'str'>  :  Tony


In [ ]:
registered_persons = [][][]

In [ ]:
print("Additional information about", registered_name)
print("")
id = input("Enter the identification: ")
print("")
birth_date = input("Birthdate: ")
print("")
person = {'name': registered_name, 'birth_date': birth_date}
registered_persons.append(person)

In [ ]:
for person in registered_persons:
  print(type(person))

<class 'dict'>


# Recognize Faces

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
bbox = ''

while True:
  js_reply = video_frame(label_html, bbox)
  if not js_reply:
    break

  # convert JS response to OpenCV Image
  frame = js_to_image(js_reply["img"])

  # Get the face locations from the frame
  face_locations = face_recognition.face_locations(frame)

  # For each face in the frame
  for (top, right, bottom, left) in face_locations:
      # Draw a rectangle around the face
      cv2.rectangle(
          frame,
          (left, top),
          (right, bottom),
          (0, 0, 255),
          2,
      )
      # Get the face encoding of the face
      face_encoding = face_recognition.face_encodings(frame, [(top, right, bottom, left)])[0]

      # Compare the face encoding to the faces in the database
      matches = face_recognition.compare_faces(list(registered_faces.values()), face_encoding)

      # If there is a match
      if True in matches:
          # Get the index of the match
          match_index = matches.index(True)

          # Get the name of the match
          match_name = list(registered_faces.keys())[match_index]

          # Draw the name of the match on the frame
          cv2.putText(
              frame,
              match_name,
              (left, top - 10),
              cv2.FONT_HERSHEY_SIMPLEX,
              1,
              (0, 0, 255),
              2,
              cv2.LINE_AA,
          )
          read_file_name(match_name)
      else:
          # Draw unknown on the frame
          cv2.putText(
              frame,
              "Unknown",
              (left, top - 10),
              cv2.FONT_HERSHEY_SIMPLEX,
              1,
              (0, 0, 255),
              2,
              cv2.LINE_AA,
          )

  # Show image
  plt.imshow(frame)
  plt.show()

# Clases


## Add professor

In [ ]:
class Profesor:
    def __init__(self, identification, name, birthday, nationality,phone_numbers, emails, address, speciality,years_in_institution,vehicle_license_plate):
        self.identification = identification
        self.name = name
        self.birthday = birthday
        self.nationality = nationality
        self.phone_numbers = phone_numbers  # diccionario
        self.emails = emails  # una lista
        self.address = address
        self.speciality = speciality
        self.years_in_institution = years_in_institution
        self.vehicle_license_plate = vehicle_license_plate

    @classmethod
    def professor_info(cls):
        indentification = input("Enter the id professor: ")
        name = input("Enter professor's name (nombre para face_recognition): ")
        birthday = input("Enter professor's birthday: ")
        nacionality = input("Enter professor's nationality: ")

        phone_numbers = {}
        phone_numbers['telephone'] = input("Enter professor's telephone numbers: ")
        phone_numbers['cellphone'] = input("Enter professor's cellphone numbers: ")

        emails = input("Email(s) professor (separados por coma): ").split(',')

        address = input("Enter professor's addres: ")
        speciality = input("Enter professor's speciality: ")

        years_in_institution = input("Enter the number of years in the institution: ")
        vehicle_license_plate = input("Enter the vehicle license plate or 'n/a' if the professor does not have a vehicle: ")

        return cls(indentification, name, birthday, nacionality, phone_numbers,emails, address,speciality, years_in_institution, vehicle_license_plate)


In [ ]:

def add_professors(archivo):
    profesores = []

    try:
        with open(archivo, 'rb') as f:
            profesores = pickle.load(f)
    except FileNotFoundError:
        pass

    while True:
        new_profesor = Profesor.professor_info()
        profesores.append({
            'identification': new_profesor.identification,
            'name': new_profesor.name,
            'birthday': new_profesor.birthday,
            'nationality': new_profesor.nationality,
            'phone_numbers': new_profesor.phone_numbers,
            'emails': new_profesor.emails,
            'address': new_profesor.address,
            'speciality': new_profesor.speciality,
            'years_in_institution': new_profesor.years_in_institution,
            'vehicle_license_plate': new_profesor.vehicle_license_plate
        })

        with open(archivo, 'wb') as f:
            pickle.dump(profesores, f)

        continuar = input("¿Desea ingresar más profesores? (y/n): ").strip().lower()
        if continuar != 'y':
            break

# Uso de la función add_professors para agregar profesores y detener cuando el usuario lo indique
add_professors('profesores.pkl')

## Add Student

In [ ]:
class Student:
    def __init__(self, identification, name, birthday, nationality,phone_numbers, emails, address,degree,scholarchip,transport):
        self.identification = identification
        self.name = name
        self.birthday = birthday
        self.nationality = nationality
        self.phone_numbers = phone_numbers  # diccionario
        self.emails = emails  # una lista
        self.address = address
        self.degree = degree
        self.scholarchip = scholarchip
        self.transport = transport


    @classmethod
    def student_info(cls):
        indentification = input("Enter the id student: ")
        name = input("Enter student's name (nombre para face_recognition): ")
        birthday = input("Enter student's birthday: ")
        nacionality = input("Enter student's nationality: ")

        phone_numbers = {}
        phone_numbers['telephone'] = input("Enter student's telephone numbers: ")
        phone_numbers['cellphone'] = input("Enter student's cellphone numbers: ")

        emails = input("Email(s) professor (separados por coma): ").split(',')

        address = input("Enter student's addres: ")
        speciality = input("Enter student's speciality: ")

        scholarchip = input("Enter the type of scholarchip, if no have put 0: ")
        transport = input("Enter the type of transport: ")

        return cls(indentification, name, birthday, nacionality, phone_numbers,emails, address,speciality, scholarchip,transport)

In [ ]:
def add_student(archivo):
    students = []

    try:
        with open(archivo, 'rb') as f:
            students = pickle.load(f)
    except FileNotFoundError:
        pass

    while True:
        new_student = Student.student_info()
        students.append({
            'identification': new_student.identification,
            'name': new_student.name,
            'birthday': new_student.birthday,
            'nationality': new_student.nationality,
            'phone_numbers': new_student.phone_numbers,
            'emails': new_student.emails,
            'address': new_student.address,
            'degree': new_student.degree,
            'scholarchip': new_student.scholarchip,
            'transport': new_student.transport
        })

        with open(archivo, 'wb') as f:
            pickle.dump(students, f)

        continuar = input("¿Do yoi want to add new student? (y/n): ").strip().lower()
        if continuar != 'y':
            break

# Uso de la función add_student para agregar  y detener
add_student('students.pkl')

## Add Miscelaneous

In [ ]:
class Miscelaneous:
    def __init__(self, identification, name, birthday, nationality,phone_numbers, emails, address,shift,gross_salary,work_sector):
        self.identification = identification
        self.name = name
        self.birthday = birthday
        self.nationality = nationality
        self.phone_numbers = phone_numbers  # diccionario
        self.emails = emails  # una lista
        self.address = address
        self.shift = shift
        self.gross_salary = gross_salary
        self.work_sector = work_sector




    @classmethod
    def miscelaneous_info(cls):
        indentification = input("Enter the id miscelaneous: ")
        name = input("Enter miscelaneous's name: ")
        birthday = input("Enter miscelaneous's birthday: ")
        nacionality = input("Enter miscelaneous's nationality: ")

        phone_numbers = {}
        phone_numbers['telephone'] = input("Enter miscelaneous's telephone numbers: ")
        phone_numbers['cellphone'] = input("Enter miscelaneous's cellphone numbers: ")

        emails = input("Email(s) miscelaneous (separados por coma): ").split(',')

        address = input("Enter miscelaneous's addres: ")
        shift = input("Enter miscelaneous's shift: ")

        gross_salary= input("Enter the gross_salary: ")
        work_sector = input("Enter the work_sector for example (A1-A9): ")

        return cls(indentification, name, birthday, nacionality, phone_numbers,emails, address,shift , gross_salary, work_sector)

In [ ]:
def add_miscelaneous(archivo):
    miscelanies = []

    try:
        with open(archivo, 'rb') as f:
            miscelanies = pickle.load(f)
    except FileNotFoundError:
        pass

    while True:
        new_miscelaneous = Miscelaneous.miscelaneous_info()
        miscelanies.append({
            'identification': new_miscelaneous.identification,
            'name': new_miscelaneous.name,
            'birthday': new_miscelaneous.birthday,
            'nationality': new_miscelaneous.nationality,
            'phone_numbers': new_miscelaneous.phone_numbers,
            'emails': new_miscelaneous.emails,
            'address': new_miscelaneous.address,
            'shift': new_miscelaneous.shift,
            'gross_salary': new_miscelaneous.gross_salary,
            'work_sector': new_miscelaneous.work_sector
        })

        with open(archivo, 'wb') as f:
            pickle.dump(miscelanies, f)

        continuar = input("¿Do you want to add new miscelaneous? (y/n): ").strip().lower()
        if continuar != 'y':
            break

# Uso de la función add_miscelaneous para agregar y detener cuando el usuario lo indique
add_miscelaneous('miscelanies.pkl')

## Add Classroom

## Add course

# Read file

In [ ]:
def read_file():
    f = open("personas.bin", "rb")
    f.seek(0)
    flag = 0
    while flag ==0:
        try:
            e = pickle.load(f)
            print(e.name)
        except:
            print("Fin")
            flag = 1
    f.close

In [ ]:
read_file()

Abel
Fin


In [ ]:
def read_file_name(name):
    f = open("personas.bin", "rb")
    f.seek(0)
    flag = 0
    while flag ==0:
        try:
            e = pickle.load(f)
            if e.name == name:
              print("Encontrado", e.name)
        except:
            print("Fin")
            flag = 1
    f.close